In [7]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression

# Wisienka na koniec
from sklearn.tree import DecisionTreeClassifier

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Warnings will not pass!
import warnings
warnings.filterwarnings('ignore')

### Importing data and initial look into it

In [ ]:
cardio_data = pd.read_csv()

### TODO: Dummy train

### TODO: We can do better meme

### TODO: Sklearn - not only for learning

### TODO: Splitting datasets - why the heck is that

### TODO: Common sence cleaning

### TODO: Encoding

#### TODO: Label encoding (Ordinal)

#### TODO: One Hot Encoding

### TODO: Scaling

### TODO: Maybe some advanced feature engineering?

### TODO: Step back meme

### TODO: Cross validation and hyperparameter tuning on SVM
Since LogisticRegression doesn't give you much hyperparameters to tune.

### TODO: Function to apply all of these things to test dataset